#Data 07: AirBnB

*  에어비앤비(Airbnb, Inc.)는 2008년 8월 시작된 세계 최대의 숙박 공유 서비스이다. 자신의 방이나 집, 별장 등 사람이 지낼 수 있는 모든 공간을 임대할 수 있다. 이 데이터는 2019년 뉴욕에서 업데이트된 에어비앤비 리스트이다.
*   Data from: http://insideairbnb.com/

##1.데이터 둘러보기

In [ ]:
#한글 폰트 설정하기
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
#런타임 다시 시작

In [ ]:
#기본 패키지 불러오기
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('seaborn')
sns.set(font_scale=1)
plt.rc('font', family='NanumBarunGothic') 
plt.rcParams['font.family'] = 'NanumGothic'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#구글 드라이브 마운트하기(이미 되어있다면 건너뛴다)
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
#데이터 불러오기
#드라이브에 올려준 csv 데이터 파일 불러오기
bnb = pd.read_csv('/content/drive/MyDrive/Python/07_Airbnb/AB_NYC_2019.csv')

In [ ]:
#불러온 데이터 일단 보기
bnb.head(3)

<컬럼 설명>   
* id: 일련번호
* name: AirBnB 이름
* host_id: 호스트 ID  
* host_name: 호스트 성명 
* neighbourhood_group: 지역   
* latitude: 위도  
* longitude: 경도  
* room_type: 방 타입  
* price: 가격  
* minimum_nights: 최소 숙박일수  
* number_of_reviews: 리뷰 개수
* last_review: 최근 리뷰 일자
* reviews_per_month: 월 평균 리뷰 수
* calculated_host_listings_count:  호스트에게 대여 가능한 에어비앤비의 총 개수
* availability_365: 1년 중 가능한 대여일수

* 범주형 변수: 
               host_name, Neighbor_group, neighbourhood, room_type
* 연속형 변수: 
               id, latitude, longitude, price, minimum_nights, number_of_reviews, last_review,
               reviews_per_month registered_host_listings_count, availability_365

In [ ]:
#데이터 기본 정보 확인하기
bnb.info()

In [ ]:
#데이터 내용 간단히 정리해서 출력하기
print("총 데이터 개수: ", bnb.shape[0]*bnb.shape[1])
print("총 결측치 수: {} = 전체 데이터의 {:.2f}% ".format(bnb.isnull().sum().sum(), (bnb.isnull().sum().sum()*100)/(bnb.shape[0]*bnb.shape[1])))
print("호스트 ID: {} 개, 호스트 {} 명".format(bnb['host_id'].nunique(), bnb['host_name'].nunique()))
print("2019년 뉴욕의 평균 Airbnb 금액: {:.2f} $".format(bnb['price'].mean()))

##2.데이터를 보고 질문 만들기

* 가장 많이 에어비앤비에 리스트 된 호스트는?
* 지역별로 가장 비싼 방과 저렴한 방은?
* 뉴욕의 어느 지역이 가장 비쌀까?
* 지역별로 에어비앤비 타입이 다를까?
* 에어비앤비 금액을 예측할 수 있을까?

##3.데이터 정비하기

In [ ]:
#컬럼별로 NULL 개수 확인하기
bnb.isnull().sum()

In [ ]:
#결측치를 간단하게 시각화 해주는 패키지 사용해보자
import missingno

In [ ]:
#matrix로 결측치 확인하기
missingno.matrix(bnb, figsize =(12,5))

In [ ]:
#bar chart로 결측치 확인하기
missingno.bar(bnb, figsize =(12,5))

In [ ]:
#NaN 값 처리하기 
bnb.fillna({'reviews_per_month':0}, inplace=True) #월 평균 리뷰 개수(reviews_per_month)에는 0 처리
bnb.fillna({'name':"NoName"}, inplace=True) #이름이 공백인 경우는 NoName으로 처리

In [ ]:
#사용하지 않을 컬럼은 삭제한다
#last review 컬럼은 대체할만한 컬럼이 있으므로 삭제해도 된다
bnb.drop(['id','last_review'], axis=1, inplace=True) 

In [ ]:
#에어비앤비 이름과 호스트 이름이 같은 것 존재(체크!)
bnb[bnb['name']==bnb['host_name']]

In [ ]:
#중복된 컬럼 확인
bnb.duplicated().sum()
#bnb.drop_duplicates(inplace=True) #중복된 컬럼이 존재한다면 삭제

In [ ]:
bnb.describe()

In [ ]:
#금액이 0인 에어비앤비가 11개 존재한다!(체크!)
len(bnb[bnb['price']==0])

In [ ]:
bnb.info()

##4.EDA & Visualization

###4-1. 간단한 분석

In [ ]:
#뉴욕 Airbnb의 금액 분포 나타내기

f,ax = plt.subplots(1, 1, figsize=(8,6))
sns.distplot(bnb['price'],hist=True, ax=ax)

ax.set_xlim(-1000, 6000) #x축 범위 조정하기
#데이터가 한쪽으로 매우 쏠려있는 것 같다!

In [ ]:
#skewness and kurtosis 구하기
#Skewness(왜도): 데이터가 정규분포에서 한 쪽으로 치우쳐있는가?
#                -3 < skewness < 3 사이의 값이면 기준에 부합한다
#Kurtosis(첨도): 얼마나 뾰족한가(데이터가 얼마나 평균 근처에 몰려있는가)?
#               : 정규분포의 kurtosis = 0
print("Skewness: %.2f" % bnb['price'].skew())
print("Kurtosis: %.2f" % bnb['price'].kurt())
#skewness와 kurtosis 모두 매우 높게 나타난다!
#kurtosis 값이 크다는 것 = 이상치가 많이 존재한다

In [ ]:
#금액과 최소 숙박일수만 남겨놓기
pd.concat([bnb['price'],bnb['minimum_nights']],axis=1)

In [ ]:
#가격(price)과 최소 숙박일수(minimum_nights) 간의 관계 확인하기 

bnb.plot.scatter(x='price',y='minimum_nights',ylim=(0,1500), s=0.9, color='red')
#가격이 0인 데이터가 많이 존재한다!
#이상치로 보이는 데이터도 많다

In [ ]:
#금액 분포를 box plot으로 확인하자
f, ax = plt.subplots(1, 1, figsize=(5, 6), dpi = 80)

#box plot
sns.boxplot(bnb['price'], showfliers = True, ax=ax)
#outlier 값이 많이 존재함을 알수 있다: 분석 시 주의!

In [ ]:
#컬럼 간 상관관계를 heatmap으로 확인하자
f, ax = plt.subplots(1, 1, figsize=(9, 8))

corrmat = bnb.corr()
sns.heatmap(corrmat, annot=True, cmap='YlGnBu', ax=ax)
plt.title('Correlation map for Airbnb Data', size=15)

#host_id와 월평균리뷰 수(reveiws_per_month), availability_365간에 상관 관계가 있다
#최소숙박일수(min_nights)와 no_of_listings_count 및 availability_365 사이에는 주목할만한 상관 관계가 있다
#가격은 availability_365 및 host_listings_count와의 상관 관계가 있다
#리뷰 수(number_of_reviews)와 월평균 리뷰 수(reviews_per_month)는 거의 동일한 정보값을 준다 >> 두 변수 중 하나로 분석하자!
#리뷰 수(number_of_reviews)는 availability_365와 관련 있다

###4-2. 금액의 threshold를 정해서 outlier 제거하기

In [ ]:
#이 분석은 데이터 정비에 들어가도 됨...

min_threshold,max_threshold= bnb['price'].quantile([0.01,0.99]) #백분위수로 0.01, 0.99에 있는 price 확인하기
min_threshold,max_threshold
#min_th

In [ ]:
bnb[bnb['price'] < min_threshold] 
#min 404개, max 474개

In [ ]:
bnb_thr = bnb[(bnb['price'] > min_threshold) & (bnb['price'] < max_threshold)]
bnb_thr.info()

In [ ]:
#sns.distplot으로 금액 분포가 어떻게 달라졌는지 확인해보자
sns.distplot(bnb_thr['price'])

In [ ]:
#skewness and kurtosis
print("Skewness: %.2f" % bnb_thr['price'].skew())
print("Kurtosis: %.2f" % bnb_thr['price'].kurt())

In [ ]:
sns.boxplot(bnb_thr['price'])

###4-3. 가장 많이 리스트 된 호스트 

In [ ]:
bnb_thr.head(3)

In [ ]:
#가장 많이 리스팅 된 호스트(이름 기준)는?
top_host= bnb_thr.host_name.value_counts().head(10)
top_host

In [ ]:
top_host2 = pd.DataFrame(top_host)
top_host2.reset_index(inplace=True)
top_host2.rename(columns={'index': 'host_name', 'host_name':'count'}, inplace=True)
top_host2

In [ ]:
#bar 그래프로 나타내기
f, ax = plt.subplots(1, 1, figsize = (12,6))

sns.barplot(x='host_name', y= 'count', data=top_host2, palette='rocket', ax=ax)
ax.set_title('NYCs top hosts with the most listings')
ax.set_xlabel('Host Name')
ax.set_ylabel('Counts')

ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

###4-4. 가장 비싼 방과 가장 저렴한 방은?

In [ ]:
#지역별로 가장 비싼 방은?
#idxmax: 최대값을 가지는 index를 출력한다
bnb_thr.groupby(['neighbourhood_group'])['price'].idxmax()

In [ ]:
#지역별로 가장 비싼 방은?
bnb_thr.loc[bnb_thr.groupby(['neighbourhood_group'])['price'].idxmax()][['name','neighbourhood_group','host_name','price']]

In [ ]:
#지역별로 가장 저렴한 방은?
#idxmax: 최대값을 가지는 index를 출력한다
bnb_thr.groupby(['neighbourhood_group'])['price'].idxmin()

In [ ]:
#지역별로 가장 저렴한 방은?
bnb_thr.loc[bnb_thr.groupby(['neighbourhood_group'])['price'].idxmin()][['name','neighbourhood_group','host_name','price']]

###4-5. 지역별 금액 분포

In [ ]:
#지역(neighbourhood_group)이 어떻게 나뉘어져 있는지 확인하기
bnb_thr['neighbourhood_group'].unique()

In [ ]:
#Violin plot으로 지역 별 에어비앤비 금액 분포 확인하기
f, ax = plt.subplots(1, 1, figsize = (12,6))

sns.violinplot(x='neighbourhood_group',y='price', data=bnb_thr ,palette='winter', ax=ax)
ax.set_title('Prices with different regions')

#맨하탄이 대체적으로 가격이 높다!
#대부분 100$ 이하에서 가격대가 형성되어 있다

###4-6. 지역별 방 타입

In [ ]:
#방 타입(room_type) 종류는?
bnb_thr['room_type'].unique()

In [ ]:
#groupby로 집계한 다음 unstack 함수를 사용해서 깔끔하게 나타낸다
room_type2 = bnb_thr.groupby(['neighbourhood_group'])['room_type'].value_counts().unstack(0)
room_type2

In [ ]:
#지역별로 어떤 방 타입이 많은가?
g = room_type2.plot(kind='bar', figsize = (12,6))
g.set_title('Diffrent room types in various regions')
g.set_xticklabels(g.get_xticklabels(), rotation=0)

#맨하탄은 아파트/집 전체를 빌려주는 형태가 가장 많음
#독립적인 방 하나만 빌려주는 것은 브루클린이 가장 많음
#쉐어룸은 개수가 가장 적다

###4-7. 리뷰가 많은 방은?

In [ ]:
#뉴욕에서 가장 리뷰가 많이 된 방
top10_review= bnb.nlargest(10,'reviews_per_month')
top10_review[['name','reviews_per_month','neighbourhood_group']]

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10,8))

sns.stripplot(x='room_type',y='reviews_per_month', data = bnb, hue='neighbourhood_group'
  , dodge=True, jitter = True, palette='Set2', ax=ax) 
#dodge = hue 별로 데이터를 나눠서 쌓아준다
#jitter = 가로축 위치에 변동을 줘서 데이터가 겹치지 않도록 해줌

ax.set_title('Most Reviewed room_types in each Neighbourhood Groups')

#개인실은 퀸즈가 가장 많은 리뷰를 많이 받았다
#맨하탄과 브루클린은 리뷰 수가 비슷하다

###4-8. 위도와 경도로 지도 위에 위치 나타내기

In [ ]:
f, ax = plt.subplots(1, 2, figsize = (18, 10))

sns.scatterplot(x= 'longitude', y='latitude', data=bnb_thr, hue= 'neighbourhood_group',palette='bright', edgecolor='black',linewidth=0.3, ax=ax[0])
sns.scatterplot(x= 'longitude', y='latitude', data=bnb_thr, hue= 'room_type',edgecolor='black',linewidth=0.3, ax=ax[1])

In [ ]:
#원의 크기로 데이터 나타내기
f, ax = plt.subplots(1, 1, figsize=(10, 8))

sns.scatterplot(data=bnb_thr, x='longitude', y='latitude', hue="availability_365", 
                palette='coolwarm', size='availability_365', sizes=(20,300))

##5.선형 회귀 분석

In [ ]:
#필요한 패키지 불러오기
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
bnb_md = bnb

In [ ]:
#범주형 변수들을 라벨인코더로 변환시켜주기
labelencoder = LabelEncoder()
bnb_md['neighbourhood_group'] = labelencoder.fit_transform(bnb_md['neighbourhood_group'])
bnb_md['neighbourhood'] = labelencoder.fit_transform(bnb_md['neighbourhood'])
bnb_md['room_type'] = labelencoder.fit_transform(bnb_md['room_type'])

In [ ]:
feature_columns  = ['neighbourhood_group', 'neighbourhood', 'room_type', 'minimum_nights',
                 'calculated_host_listings_count', 'availability_365']

In [ ]:
#설명변수
X = bnb_md[feature_columns]
#타겟변수
y = bnb_md['price']

#학습 데이터와 평가 데이터 나누기(7:3으로 나누자!)
#train: 학습 데이터, test: 평가 데이터
train_x, test_x, train_y, test_y = train_test_split(X, y, train_size=0.7,test_size=0.3,random_state=42)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

In [ ]:
# 선형회귀분석
linreg = LinearRegression()
linreg.fit(train_x, train_y)

In [ ]:
#결과 확인하기
test_predict = linreg.predict(test_x) 
print('Accuracy on test set: {}'.format(linreg.score(test_x, test_y)))

In [ ]:
#결과 확인하기 2
test_predict = linreg.predict(test_x) 
error = pd.DataFrame(np.array(test_y).flatten(),columns=['Actual'])
error['Prediction'] = np.array(test_predict)
error['Delta'] = abs(error['Actual'] - error['Prediction'])
error.head(10)

##6.Review

* Airbnb 데이터는:  
> 총 데이터 개수:  782320  
총 결측치 수: 20141 = 전체 데이터의 2.57%   
호스트 ID: 37457 개, 호스트 11452 명  
2019년 뉴욕의 평균 Airbnb 금액: 152.72 $  
* 가장 많이 에어비앤비에 리스트 된 호스트는?
* 지역별로 가장 비싼 방과 저렴한 방은?
* 뉴욕의 어느 지역이 가장 비쌀까?
* 지역별로 에어비앤비 타입이 다를까?
* 에어비앤비 금액을 예측할 수 있을까?:  
> 이 데이터의 경우 다른 컬럼을 이용해 선형회귀로 금액을 예측하기 어려웠다.   
데이터 분석이 중요한 이유!

